In [1]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
import numpy as np

2022-12-08 07:49:23.825921: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 07:49:23.966354: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-08 07:49:23.970381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 07:49:23.970396: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# test follow tut-doc
schema = tfgnn.read_schema('./tfgnn_test/schema_test.pbtxt')

In [12]:
# (issue proposed)
# with tf.io.TFRecordWriter('./tfgnn_test/dummy/try_write.tfrecord') as writer:
#     for _ in range(10):
#         graph = tfgnn.random_graph_tensor(schema)
#         example = tfgnn.write_example(graph)
#         writer.write(example.SerializeToString())

In [14]:
with tf.io.TFRecordWriter('./tfgnn_test/dummy/try_write.tfrecord') as writer:
    for _ in range(1):
        spec = tfgnn.create_graph_spec_from_schema_pb(schema)
        graph = tfgnn.random_graph_tensor(spec)
        example = tfgnn.write_example(graph)
        writer.write(example.SerializeToString())

In [17]:
graph

GraphTensor(
  context=Context(features={'label_isBP': <tf.Tensor: shape=(1,), dtype=tf.int64>, 'bytes_input_rate': <tf.Tensor: shape=(1,), dtype=tf.float32>, 'records_input_rate': <tf.Tensor: shape=(1,), dtype=tf.float32>}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['source', 'operate'],
  edge_set_names=['op2op', 'src2op'])

### here we try to make a hand-made in-memory GraphTensor instance

In [19]:
# first review the schema
schema

context {
  features {
    key: "bytes_input_rate"
    value {
      description: "Bytes input rate from the source vertex."
      dtype: DT_FLOAT
    }
  }
  features {
    key: "label_isBP"
    value {
      description: "Label: whether job back-pressured."
      dtype: DT_INT64
    }
  }
  features {
    key: "records_input_rate"
    value {
      description: "Records input rate from the source vertex."
      dtype: DT_FLOAT
    }
  }
}
node_sets {
  key: "operate"
  value {
    description: "The non-source vertex."
    features {
      key: "bytes_selectivity"
      value {
        description: "Vertex\342\200\231s selectivity for out/in-bytes."
        dtype: DT_FLOAT
      }
    }
    features {
      key: "parallelism"
      value {
        description: "Vertex\342\200\231s set parallelism."
        dtype: DT_INT64
      }
    }
    features {
      key: "records_selectivity"
      value {
        description: "Vertex\342\200\231s selectivity for out/in-records."
        dtype:

In [95]:
graph = tfgnn.GraphTensor.from_pieces(
    node_sets={
        'source': tfgnn.NodeSet.from_fields(
            sizes=tf.constant([1]),
            features={
                'parallelism': tf.constant([4,])
            }),
        'operate': tfgnn.NodeSet.from_fields(
            sizes=tf.constant([2]),
            features={
                'bytes_selectivity': tf.constant([0.9, 0.0,]),
                'records_selectivity': tf.constant([0.1, 0.0,]),
                'parallelism': tf.constant([4, 4,]),
                'utilization_embedding': tf.constant([[1,2,3,4,5,6,],[6,5,4,3,2,1,]]),
            }),
    },
    edge_sets={
        'src2op': tfgnn.EdgeSet.from_fields(
            sizes=tf.constant([1]),
            adjacency=tfgnn.Adjacency.from_indices(
                source=('source', tf.constant([0,])),
                target=('operate', tf.constant([0,])))),

#         'op2op': tfgnn.EdgeSet.from_fields(
#             sizes=tf.constant([1]),
#             adjacency=tfgnn.Adjacency.from_indices(
#                 source=('operate', tf.constant([0,])),
#                 target=('operate', tf.constant([1,])))),

        ### test what if no op2op edge:
            'op2op': tfgnn.EdgeSet.from_fields(
            sizes=tf.constant([0]),
            adjacency=tfgnn.Adjacency.from_indices(
                source=('operate', tf.constant([], dtype=tf.int64)),
                target=('operate', tf.constant([], dtype=tf.int64)))),
        ### test result: no err raised

    },
    context=tfgnn.Context.from_fields(
            features={
                'bytes_input_rate': tf.constant([200,]),
                'records_input_rate': tf.constant([20,]),
                'label_isBP': tf.constant([1]),
            })
)

In [96]:
# graph.spec

In [97]:
with tf.io.TFRecordWriter('./tfgnn_test/dummy/try_write.tfrecord') as writer:
    example = tfgnn.write_example(graph)
    print(example)
#     writer.write(example.SerializeToString())

features {
  feature {
    key: "context/bytes_input_rate"
    value {
      int64_list {
        value: 200
      }
    }
  }
  feature {
    key: "context/label_isBP"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "context/records_input_rate"
    value {
      int64_list {
        value: 20
      }
    }
  }
  feature {
    key: "edges/op2op.#size"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "edges/op2op.#source"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "edges/op2op.#target"
    value {
      int64_list {
      }
    }
  }
  feature {
    key: "edges/src2op.#size"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "edges/src2op.#source"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "edges/src2op.#target"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "nodes/o

### here we try to implement the GraphTensor maker

In [83]:
embeddings.shape, len(keys)

((22, 6), 22)

In [102]:
# pseudo data
embeddings = np.array([[ 3.57395299e-02,  8.06235150e-03, -3.74955907e-02,
        -3.39482054e-02,  1.23462565e-02, -1.59011856e-02],
       [ 1.13631797e+00, -8.71300697e+00, -1.53374147e+01,
        -2.91174412e+01, -1.15212708e+01, -1.00926161e+01],
       [ 4.74400806e+00,  1.45451097e+01, -3.14076691e+01,
        -1.09569511e+01,  1.78568554e+01, -9.63525963e+00],
       [ 2.18789787e+01, -5.58547354e+00,  9.31024742e+00,
         2.55261765e+01,  2.58095055e+01, -1.60838947e+01],
       [-4.02473211e+00, -2.86237001e+00, -2.35737762e+01,
        -1.08697271e+01, -2.11060047e+01, -1.05461378e+01],
       [ 1.07403269e+01,  2.78513813e+01, -4.96649208e+01,
        -1.36362076e+01,  2.01553040e+01, -4.45996141e+00],
       [-2.72763801e+00, -7.94400358e+00,  3.45084229e+01,
         2.99869728e+01,  3.94443178e+00, -9.77673531e+00],
       [-5.55253744e+00, -3.76001549e+00, -2.15469303e+01,
        -1.03292923e+01, -1.79221001e+01, -1.14029608e+01],
       [-1.49511995e+01, -5.74085093e+00, -1.35172186e+01,
        -3.00543041e+01, -7.92033863e+00, -1.16169939e+01],
       [ 2.81525135e+01, -9.28903770e+00, -3.20226240e+00,
         2.87621555e+01,  2.52876778e+01, -1.40687141e+01],
       [ 3.96669126e+00, -6.40941238e+00,  1.51890516e+01,
         5.13268518e+00,  1.93375664e+01, -1.92107430e+01],
       [ 5.56613743e-01, -7.33412218e+00, -3.16105080e+01,
        -1.40403166e+01, -1.71877136e+01, -1.34753399e+01],
       [-3.89417648e+01,  3.53432268e-01, -2.09219856e+01,
        -2.29946365e+01, -3.46543908e+00, -1.43928156e+01],
       [-9.91747952e+00,  4.01413441e+00, -6.68075657e+00,
        -3.79401231e+00, -2.38905773e+01, -3.48761511e+00],
       [ 5.50344801e+00, -1.62924538e+01, -2.97262840e+01,
        -2.59126968e+01, -1.15528898e+01, -2.16320248e+01],
       [-2.47912846e+01, -1.00922127e+01, -2.83760948e+01,
        -2.82528152e+01,  8.10468960e+00, -1.74723625e+01],
       [-8.24438381e+00, -7.75321865e+00,  4.25156593e+00,
        -1.33596029e+01, -2.22319984e+01, -1.87092323e+01],
       [ 1.17066078e+01, -6.87926340e+00, -1.29598494e+01,
        -6.76033020e+00,  2.34065475e+01, -1.90525017e+01],
       [-4.49013233e+00, -1.63534107e+01,  3.00725708e+01,
         1.63671646e+01,  1.59018822e+01, -1.44117346e+01],
       [ 1.57391024e+01, -1.23704939e+01,  3.80741143e+00,
         1.42136726e+01,  3.15935974e+01, -1.49392471e+01],
       [-4.67871571e+00, -4.61830807e+00,  9.04006672e+00,
        -3.76126242e+00, -2.62182541e+01, -1.44378605e+01],
       [ 2.03958874e+01,  1.51550055e+01, -2.60176182e+01,
        -1.07382650e+01,  1.63624554e+01, -8.23231506e+00]],)

keys = ['', 'q4v1', 'q4v2', 'q4v3', 'q5v1', 'q5v2', 'q5v3', 'q7v2', 'q9v1', 'q9v2', 'q3v1', 'q8v1', 'q8v2', 'q8v3', 'q11v1', 'q12v1', 'q16v1', 'q17v1', 'q18v1', 'q19v1', 'q20v1', 'q7v1']

row = {'qid': 4, 'pseudoInputBytesRate': 6.826667e+04, 'pseudoInputRecordsRate': 1.677400e+03, 'vPs': '[2,2,2,2]', 'isBP': False }

DAG = {4: [[0,1,2,],[1,2,3]]}

selectivities = {'true_selectivity_records': {(3, 1): 0.0,
  (4, 1): 0.6912518503136663,
  (4, 2): 0.30498580132267283,
  (4, 3): 0.0,
  (5, 1): 0.41727981517418555,
  (5, 2): 0.0002818181698208069,
  (5, 3): 0.0,
  (7, 1): 1.4371688367287183e-06,
  (7, 2): 0.0,
  (8, 1): 0.7527639835282449,
  (8, 2): 0.13995420932258765,
  (8, 3): 0.0,
  (9, 1): 0.6910047481349646,
  (9, 2): 0.0,
  (10, 1): 0.0,
  (11, 1): 0.0,
  (12, 1): 0.0,
  (15, 1): 0.0,
  (16, 1): 0.0,
  (17, 1): 0.0,
  (18, 1): 0.0,
  (19, 1): 0.0,
  (20, 1): 0.0},
 'true_selectivity_bytes': {(3, 1): 0.0,
  (4, 1): 0.6753447790534584,
  (4, 2): 0.24189611914570447,
  (4, 3): 0.0,
  (5, 1): 0.561559493159304,
  (5, 2): 0.05540181310893824,
  (5, 3): 0.0,
  (7, 1): 8.078045795509282e-05,
  (7, 2): 0.0,
  (8, 1): 0.7724989981804398,
  (8, 2): 0.14459429349535358,
  (8, 3): 0.0,
  (9, 1): 1.9174096940261498,
  (9, 2): 0.0,
  (10, 1): 0.0,
  (11, 1): 0.0,
  (12, 1): 0.0,
  (15, 1): 0.0,
  (16, 1): 0.0,
  (17, 1): 0.0,
  (18, 1): 0.0,
  (19, 1): 0.0,
  (20, 1): 0.0}}

In [139]:
def make_graph_tensor(**kwargs):
    # function for job-level 
    
    nodes = kwargs['nodes']     # {source: [parallelisms], 
                                #  operate: [parallelisms, 
#                                            bytes_selectivities,
#                                            records_selectivities,
#                                            utilization_embeddings]}
    edges = kwargs['edges']     # {[[sources,],[targets,]]}, {src2op, op2op}
    context = kwargs['context'] # [3], [bytes_inpute_rate, records_input_rate, label_isBP,]
    
    graph = tfgnn.GraphTensor.from_pieces(
        # make node_sets
        node_sets={
            'source': tfgnn.NodeSet.from_fields(
                sizes=tf.constant([len(nodes['source'])]),
                features={
                    'parallelism': tf.constant(nodes['source'])
                }),
            'operate': tfgnn.NodeSet.from_fields(
                sizes=tf.constant([len(nodes['operate'][0])]),
                features={
                    'parallelism': tf.constant(nodes['operate'][0]),
                    'bytes_selectivity': tf.constant(nodes['operate'][1]),
                    'records_selectivity': tf.constant(nodes['operate'][2]),
                    'utilization_embedding': tf.constant(nodes['operate'][3]),
                }),
        },
        # make edge_sets
        edge_sets={
            'src2op': tfgnn.EdgeSet.from_fields(
                sizes=tf.constant([len(edges['src2op'][0])]),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=('source', tf.constant(edges['src2op'][0])),
                    target=('operate', tf.constant(edges['src2op'][1])))),
            'op2op': tfgnn.EdgeSet.from_fields(
                sizes=tf.constant([len(edges['op2op'][0])]),
                adjacency=tfgnn.Adjacency.from_indices(
                    source=('operate', tf.constant(edges['op2op'][0])),
                    target=('operate', tf.constant(edges['op2op'][1])))),
        },
        # make context
        context=tfgnn.Context.from_fields(
                features={
                    'bytes_input_rate': tf.constant([context[0],]),
                    'records_input_rate': tf.constant([context[1],]),
                    'label_isBP': tf.constant([context[2]]),
                })
    )

    return graph

In [143]:
class graph_tensor_dataset_maker:
    def __init__(self, embeddings, keys, DAG, selectivities, o_fp='./tfgnn_test/dummy/try_write.tfrecord'):
        self.embeddings = embeddings
        self.keys = list(keys) # ndarray -> list
        self.DAG = DAG
        self.selectivities = selectivities
        self.o_fp = o_fp
    
    def single_writer(self, graph, demo=True):
        with tf.io.TFRecordWriter(self.o_fp) as writer:
            example = tfgnn.write_example(graph)
            if demo:
                print(example)
            writer.write(example.SerializeToString())
        
#     row = {'qid': 4, 'pseudoInputBytesRate': 0.9, 'pseudoInputRecordsRate': 0.1, 'vPs': '[2,2,2,2]', 'isBP': False }
    def make(self, row):
        # encode sample row data
        qid = row['qid']
        inBrate = row['pseudoInputBytesRate']
        inRrate = row['pseudoInputRecordsRate']
        parallels = eval(row['vPs'])
        label = row['isBP']
        vnum = len(parallels)
        
        # find coress-information        
        dag = self.DAG[qid]
        embeds = [] # length should be vnum-1, since no embedding for the source vertex
        for i in range(1, vnum):
            vkeyname = ''.join(['q', str(qid), 'v', str(i)])
            embeds.append(self.embeddings[self.keys.index(vkeyname)])
        
        # make context
        context = [row['pseudoInputBytesRate'],
                   row['pseudoInputRecordsRate'],
                   1 if row['isBP'] else 0]
        
        # make edges
#         edges = {'src2op':[], 'op2op':[],}
        src2op_edges = []
        op2op_edges = []
        for i in range(len(dag[0])):
            s = dag[0][i]
            t = dag[1][i]
            if s == 0:
                src2op_edges.append([s,t])
            else:
                op2op_edges.append([s,t])
        edges = {
            'src2op': [[e[0] for e in src2op_edges],
                       [e[1] for e in src2op_edges]],
            'op2op': [[e[0]-1 for e in op2op_edges],
                      [e[1]-1 for e in op2op_edges]],
        }
        
        # make nodes
        nodes = {}
        nodes['source'] = [parallels[0]]
        vselects = []
        for i in range(1, vnum):
            vselects.append([
                self.selectivities['true_selectivity_bytes'][(qid, i)],
                self.selectivities['true_selectivity_records'][(qid, i)],
            ])
        nodes['operate'] = [
            parallels[1:],
            [e[0] for e in vselects],
            [e[1] for e in vselects],
            embeds,
        ]
        
        # convert to tfgnn graph tensor
        graph = make_graph_tensor(nodes=nodes, edges=edges, context=context)
        
        # write it down
        self.single_writer(graph)

In [144]:
amaker = graph_tensor_dataset_maker(embeddings, keys, DAG, selectivities,)

In [145]:
amaker.make(row)

features {
  feature {
    key: "context/bytes_input_rate"
    value {
      float_list {
        value: 68266.671875
      }
    }
  }
  feature {
    key: "context/label_isBP"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "context/records_input_rate"
    value {
      float_list {
        value: 1677.4000244140625
      }
    }
  }
  feature {
    key: "edges/op2op.#size"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "edges/op2op.#source"
    value {
      int64_list {
        value: 0
        value: 1
      }
    }
  }
  feature {
    key: "edges/op2op.#target"
    value {
      int64_list {
        value: 1
        value: 2
      }
    }
  }
  feature {
    key: "edges/src2op.#size"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "edges/src2op.#source"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "edges/src2op.#target"
  

In [114]:
keys.index('q4v1')

1

In [134]:
edges = {'op2op':[[0, 1], [1, 2]]}

tfgnn.EdgeSet.from_fields(
    sizes=tf.constant([len(edges['op2op'][0])]),
    adjacency=tfgnn.Adjacency.from_indices(
    source=('operate', tf.constant(edges['op2op'][0])),
    target=('operate', tf.constant(edges['op2op'][1]))))

EdgeSet(features={}, sizes=[2], adjacency=Adjacency(source=('operate', <tf.Tensor: shape=(2,), dtype=tf.int32>), target=('operate', <tf.Tensor: shape=(2,), dtype=tf.int32>)))

In [ ]:
tfgnn.create_graph_spec_f